# RAGAS 를 활용한 평가

**참고**
- RAGAS: https://docs.ragas.io/en/latest/getstarted/evaluation.html

아래의 주석을 해제한 후 실행하여 패키지를 설치 후 진행해주세요

In [ ]:
# !pip install -qU faiss-cpu ragas

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

### 저장한 CSV 파일로부터 로드

- `data/ragas_synthetic_dataset.csv` 파일을 로드합니다.

In [ ]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df

In [ ]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

In [ ]:
import ast


# contexts 컬럼의 문자열을 리스트로 변환
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}


test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

In [ ]:
test_dataset[1]["contexts"]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

배치 데이터셋을 생성합니다. 배치 데이터셋은 다량의 질문을 한 번에 처리할 때 용이합니다.

- 배치: https://wikidocs.net/233345

In [ ]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

`batch()` 를 호출하여 배치 데이터셋에 대한 답변을 받습니다.

In [ ]:
answer = chain.batch(batch_dataset)
answer[:3]

LLM 이 생성한 답변을 'answer' 컬럼에 저장합니다.

In [ ]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

## 답변 평가

### Context Recall

**요약** 

- "검색된 context가 LLM 이 생성한 답변과 얼마나 일치하는지" 를 측정합니다.

Context recall은 검색된 context가 LLM 이 생성한 답변과 얼마나 일치하는지를 측정합니다. 

이는 question, ground truth 및 검색된 context를 사용하여 계산되며, 값은 0에서 1 사이로, 높을수록 더 나은 성능을 나타냅니다. 

Ground truth 답변에서 context recall을 추정하기 위해, ground truth 답변의 각 주장이 검색된 context에 귀속될 수 있는지 분석됩니다. 이상적인 시나리오에서는 ground truth 답변의 모든 주장이 검색된 context에 귀속될 수 있어야 합니다. 

$$\text{context recall} = \frac{|\text{GT claims that can be attributed to context}|}{|\text{Number of claims in GT}|}$$

### Context Precision

**요약**

- "얼마나 관련성 있는 문서가 상위에 배치되었는가?" 를 평가하는 지표입니다. 

Context Precision은 contexts 내의 ground-truth 관련 항목들이 상위 순위에 있는지를 평가하는 지표입니다. 이상적으로는 모든 관련 chunks가 상위 순위에 나타나야 합니다. 이 지표는 question, ground_truth, 그리고 contexts를 사용하여 계산되며, 0에서 1 사이의 값을 가집니다. 높은 점수일수록 더 나은 정밀도를 나타냅니다.

Context Precision@K의 계산식은 다음과 같습니다.

$$\text{Context Precision@K} = \frac{\sum_{k=1}^{K} (\text{Precision@k} \times v_k)}{\text{Total number of relevant items in the top K results}}$$

여기서 Precision@k는 다음과 같이 계산됩니다.

$$\text{Precision@k} = \frac{\text{true positives@k}}{(\text{true positives@k + false positives@k})}$$

K는 contexts의 총 chunk 수이며, $v_k \in \{0, 1\}$은 순위 k에서의 관련성 지표입니다.

이 지표는 정보 검색 시스템에서 검색된 컨텍스트의 품질을 평가하는 데 사용됩니다. 관련 정보가 얼마나 정확하게 상위 순위에 배치되었는지를 측정함으로써 시스템의 성능을 판단할 수 있습니다.

### Answer Relevancy

- "생성된 답변이 주어진 prompt에 얼마나 적절한지" 를 평가하는 지표입니다. 

이 지표의 주요 특징과 계산 방법을 요약하면 다음과 같습니다.

1. 목적: 생성된 답변의 관련성을 평가합니다.
2. 점수 해석: 낮은 점수는 불완전하거나 중복 정보를 포함한 답변을, 높은 점수는 더 나은 관련성을 나타냅니다.
3. 계산에 사용되는 요소: question, context, answer

Answer Relevancy의 계산 방법:
- 원래 question과 answer를 기반으로 생성된 합성 질문들 간의 평균 코사인 유사도로 정의됩니다.
- 수식:

$$\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \cos(E_{g_i}, E_o)$$

또는

$$\text{answer relevancy} = \frac{1}{N} \sum_{i=1}^N \frac{E_{g_i} \cdot E_o}{\|E_{g_i}\| \|E_o\|}$$

여기서:
- $E_{g_i}$는 생성된 질문 $i$의 임베딩
- $E_o$는 원래 질문의 임베딩
- $N$은 생성된 질문의 수 (기본값 3)

주의사항:
- 실제로는 점수가 대부분 0과 1 사이에 있지만, 코사인 유사도의 특성상 수학적으로 -1에서 1 사이의 값을 가질 수 있습니다.

이 지표는 질문-답변 시스템의 성능을 평가하는 데 유용하며, 특히 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지를 측정합니다.

### Faithfulness

- "생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정" 하는 지표입니다. 

주요 특징은 다음과 같습니다.

1. 목적: 답변의 사실적 일관성을 컨텍스트와 비교하여 평가합니다.
2. 계산 요소: 답변과 검색된 컨텍스트를 사용합니다.
3. 점수 범위: 0에서 1 사이로 조정되며, 높을수록 더 좋습니다.

Faithfulness 점수 계산 방법:

$$\text{Faithfulness score} = \frac{|\text{Number of claims in the generated answer that can be inferred from given context}|}{|\text{Total number of claims in the generated answer}|}$$

계산 과정:
1. 생성된 답변에서 주장(claims)들을 식별합니다.
2. 각 주장을 주어진 컨텍스트와 대조 검증하여 컨텍스트에서 추론 가능한지 확인합니다.
3. 위 수식을 사용하여 점수를 계산합니다.

예시:
- 질문: "아인슈타인은 어디서, 언제 태어났나요?"
- 컨텍스트: "알버트 아인슈타인(1879년 3월 14일 출생)은 독일 출신의 이론 물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 여겨집니다."
- 높은 충실도 답변: "아인슈타인은 1879년 3월 14일 독일에서 태어났습니다."
- 낮은 충실도 답변: "아인슈타인은 1879년 3월 20일 독일에서 태어났습니다."

이 지표는 생성된 답변이 주어진 컨텍스트에 얼마나 충실한지를 평가하는 데 유용하며, 특히 질문-답변 시스템의 정확성과 신뢰성을 측정하는 데 중요합니다.

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

In [ ]:
result_df = result.to_pandas()
result_df.head()

In [ ]:
result_df.to_csv("data/ragas_evaluation_result.csv", index=False)

In [ ]:
result_df.loc[:, "context_precision":"context_recall"]